In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import urllib.request as req
import requests
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import numpy as np
import urllib.parse as par

In [2]:

#TODO : 도쿄 치안 데이터 : 치안 관련 키워드로 일본 도쿄 공공 데이터 수집
#* 치안, 범죄, 경찰, 범죄 예방, CCTV, 범죄자, 안전, 교통 안전, 경비원, 범죄 통계
#* 비슷한 키워드로 중복 데이터 다운 가능성으로 중복 데이터 제거
searchKeywordList = ['治安','犯罪', '警察', '犯罪予防', '監視', '犯罪者', '安全', '交通安全', '警備員', '犯罪統計']


In [3]:
keyword = '治安'

In [4]:
def encoded_text(keyword):
    encoded = par.quote(keyword)
    return encoded

In [5]:
import requests
from bs4 import BeautifulSoup

def get_page_info(keyword):
    #* 첫 페이지 요청
    encoded_keyword = encoded_text(keyword)
    url = f'https://catalog.data.metro.tokyo.lg.jp/dataset?q={encoded_keyword}&sort=score+desc%2C+metadata_modified+desc'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    #* 해당 키워드 전체 페이지 수
    totalPageNum = len(soup.select('ul.pagination > li')) - 1
    #print(f"{keyword}의 전체 페이지 수:", totalPageNum)
    return totalPageNum

In [6]:
get_page_info(keyword)

3

In [7]:
def total_connect_url(keyword):
    wholeurl = []
    for idx in range(1, get_page_info(keyword) + 1):
        encoded_keyword = encoded_text(keyword)
        url = f'https://catalog.data.metro.tokyo.lg.jp/dataset?q={encoded_keyword}&sort=score+desc%2C+metadata_modified+desc&page={idx}'
        wholeurl.append(url)
    return wholeurl

In [8]:
total_connect_url(keyword)

['https://catalog.data.metro.tokyo.lg.jp/dataset?q=%E6%B2%BB%E5%AE%89&sort=score+desc%2C+metadata_modified+desc&page=1',
 'https://catalog.data.metro.tokyo.lg.jp/dataset?q=%E6%B2%BB%E5%AE%89&sort=score+desc%2C+metadata_modified+desc&page=2',
 'https://catalog.data.metro.tokyo.lg.jp/dataset?q=%E6%B2%BB%E5%AE%89&sort=score+desc%2C+metadata_modified+desc&page=3']

In [9]:
datanumList = []
def get_connect_url(keyword):
    for url in total_connect_url(keyword):
        code = requests.get(url)
        soup = BeautifulSoup(code.text, 'html.parser')
        datanum = soup.select('ul.dataset-list.list-unstyled h3.dataset-heading > a')
        connectBaseurl = 'https://catalog.data.metro.tokyo.lg.jp'
        for idx in range(len(datanum)):
            datanumList.append(connectBaseurl + datanum[idx].attrs['href'])
    #print(len(datanumList))
    return datanumList

In [10]:
len(get_connect_url(keyword))

57

In [11]:
get_connect_url(keyword)

['https://catalog.data.metro.tokyo.lg.jp/dataset/t132063d0000000011',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000007d1900000004',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d0000000002',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d1800000005',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d1800000015',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d1700000008',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d1800000018',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d1700000010',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d0000000008',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d1800000004',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d1800000009',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d0000000009',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d0000000015',
 'https://catalog.data.metro.tokyo.lg.jp/dataset/t000002d0000000016',
 'https://catalog.da

In [12]:
import os 
if not os.path.exists('tokyo/'):
    os.makedirs('tokyo/')

In [13]:
# import chardet

# with open('tokyo/fuc22a12pol01police.csv', 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(10000))

# # check what the character encoding might be
# print(result)

In [14]:
def get_csv_download_url(keyword):
    for idx in tqdm_notebook(range(len(get_connect_url(keyword)))):
        url = get_connect_url(keyword)[idx]
        code = requests.get(url, timeout=10)
        soup = BeautifulSoup(code.text, 'html.parser')
        downloadList = soup.select('ul.resource-list > li.resource-item ul.dropdown-menu li:nth-child(2) > a')
        for idx in range(len(downloadList)):
            try:
                if downloadList[idx].attrs['href'].endswith('.csv'):
                    file_name = downloadList[idx].attrs["href"].split("/")[-1]
                    pd.read_csv(downloadList[idx].attrs['href'], encoding='SHIFT_JIS').to_csv('tokyo/' + file_name,encoding='SHIFT_JIS')
            except:
                print('download 불가')
